## Setup and Data Import

In [1]:
import sys
sys.path.insert(0, '..')

from joblib import load

import Functions as fxns
from Functions import np, pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from datetime import timedelta

## Claims DF

In [2]:
# !python ../Preprocessing.py

In [3]:
claims = load('../claims.pkl')

### Pre-processing

In [ ]:
# drop ChronicCond_ prefix from applicable cols
fxns.drop_chronic_prefix(claims)

### New Columns

In [ ]:
claims['AgeAtService'] = ((claims.ClaimStartDt - claims.DOB)
                              / timedelta(days=365)).astype(int)
claims['HasDied'] = claims.DOD.notna()

physician_cols = \
    claims.columns[claims.columns.str.contains('Physician')].to_list()
claims['HasAnyPhysician'] = claims[physician_cols].notna().any(axis=1)
claims['HasAllPhysicians'] = claims[physician_cols].notna().all(axis=1)

claims['ClaimDuration'] = \
    (claims.ClaimEndDt  - claims.ClaimStartDt).dt.days + 1
claims['IPDuration'] = \
    (claims.DischargeDt - claims.AdmissionDt).dt.days + 1

outpatient_claims = claims.loc[claims.IsOutpatient == 1]
inpatient_claims  = claims.loc[claims.IsOutpatient == 0]
claims['IPClaimDuration'] = \
    (inpatient_claims.ClaimEndDt
     - inpatient_claims.ClaimStartDt).dt.days + 1
claims['OPClaimDuration'] = \
    (outpatient_claims.ClaimEndDt
     - outpatient_claims.ClaimStartDt).dt.days + 1

claims['ClaimCost'] = \
    claims.InscClaimAmtReimbursed + claims.DeductibleAmtPaid
claims['PercInsCovered'] = \
    round((claims.InscClaimAmtReimbursed / claims.ClaimCost) * 100)
claims['DailyClaimCost'] = \
    round(claims.ClaimCost / claims.ClaimDuration)

In [4]:
# encode bool colums to 0/1
fxns.re_encode_bool(claims,
    ['HasDied', 'HasAnyPhysician', 'HasAllPhysicians'])

KeyError: 'HasDied'

In [ ]:
# add date cols containing only day, week, year for each column
fxns.split_date(claims, ['ClaimStartDt', 'ClaimEndDt',
                         'AdmissionDt', 'DischargeDt'])

In [ ]:
# number of claims per beneficiary
claims = fxns.add_count_per_col(
    claims, 'BeneID', 'ClaimID', 'ClaimsPerBene')

# number of claims per attending
claims = fxns.add_count_per_col(
    claims, 'AttendingPhysician', 'ClaimID', 'ClaimsPerAttendingPhysician')

# number of claims per operating
claims = fxns.add_count_per_col(
    claims, 'OperatingPhysician', 'ClaimID', 'ClaimsPerOperatingPhysician')

# number of claims per otherphys
claims = fxns.add_count_per_col(
    claims, 'OtherPhysician', 'ClaimID', 'ClaimsPerOtherPhysician')

In [ ]:
def add_summary_col(groupby_col, mean_col, newcol1, newcol2):
    outpatient_claims = claims.loc[claims.IsOutpatient == 1]
    inpatient_claims  = claims.loc[claims.IsOutpatient == 0]
    
    ip = fxns.add_mean_per_col(
        inpatient_claims, groupby_col, mean_col, newcol1)
    op = fxns.add_mean_per_col(
        outpatient_claims, groupby_col, mean_col, newcol2)
    
    expanded_claims = pd.concat([ip, op])
    return expanded_claims

claims = add_summary_col('BeneID', 'InscClaimAmtReimbursed',
                         'Reimb_perIP', 'Reimb_perOP')
claims = add_summary_col('BeneID', 'DeductibleAmtPaid',
                         'Deductible_perIP', 'Deductible_perOP')
claims = add_summary_col('BeneID', 'PercInsCovered',
                         'PercInsCovered_perIP', 'PercInsCovered_perOP')
claims = add_summary_col('BeneID', 'DailyClaimCost',
                         'DailyClaimCost_perIP', 'DailyClaimCost_perOP')

chronic_cols = \
    claims.columns[claims.columns.str.contains('Chronic')].to_list()
for col in chronic_cols:
    claims = \
        add_summary_col('BeneID', col, f'{col}_perIP', f'{col}_perOP')

# # Mean chronic conditions for inpatients/outpatients

# # Number of states for inpatient/outpatient
# inpatient_claims = fxns.add_count_per_col(
#     inpatient_claims, 'BeneID', 'State', 'State_perIP')
# outpatient_claims = fxns.add_count_per_col(
#     outpatient_claims, 'BeneID', 'State', 'State_perOP')
# claims = pd.concat([inpatient_claims, outpatient_claims])

### More Pre-processing

In [ ]:
# change applicable cols to dtype category
fxns.to_category_dtype(claims)

### Variables

In [ ]:
# lists:
numeric_cols     = fxns.cols_by_dtype(claims)[0]
categorical_cols = fxns.cols_by_dtype(claims)[1]
date_cols        = fxns.cols_by_dtype(claims)[2]
physician_cols   = \
    claims.columns[claims.columns.str.contains('Physician')].to_list()
chronic_cols     = \
    claims.columns[claims.columns.str.contains('Chronic')].to_list()
diagnosis_cols   = \
    claims.columns[claims.columns.str.contains('Diagnosis')].to_list()
procedure_cols   = \
    claims.columns[claims.columns.str.contains('Procedure')].to_list()

# dataframes:
outpatient_claims = claims.loc[claims.IsOutpatient == 1]
inpatient_claims  = claims.loc[claims.IsOutpatient == 0]

In [ ]:
# print(claims.columns, '\n')

# Can't use, but have IPDuration count/mean/sum:
#     'AdmissionDt', 'AdmissionDt_Week',
#     'DischargeDt', 'DischargeDt_Week'

# Can't use, but have ClaimDuration count/mean/sum:
#     'ClaimEndDt', 'ClaimEndDt_Week', 'ClaimID'
#     'ClaimStartDt', 'ClaimStartDt_Week',


# Can't use, but have AgeAtService mean/sum and HasDied sum/ratio:
#     'DOB', 'DOD'

### Summary Data

In [ ]:
outpatient_claims = claims.loc[claims.IsOutpatient == 1]
inpatient_claims  = claims.loc[claims.IsOutpatient == 0]

In [ ]:
groupby_providers   = claims.groupby('Provider')

In [ ]:
mean_by_provider    = \
    groupby_providers.mean().round(2).add_suffix('_mean').reset_index()

In [ ]:
# need conditions by ratio, not count
chronic_by_provider = claims[['Provider'] + chronic_cols
    ].groupby('Provider').sum().add_suffix('_count').reset_index()

In [ ]:
providers = pd.merge(mean_by_provider, chronic_by_provider, on='Provider')
providers.sort_index(axis=1, inplace=True)

In [ ]:
print(providers.shape, '\n')
print(providers.columns)
providers.sample(5)

In [ ]:
chronic_cols

## New Columns

### Beneficiaries

In [ ]:
# providers['IPClaims_count'] = \
#     inpatient_claims.groupby('Provider').IsOutpatient.count().values
# providers['OPClaims_count'] = \
#     outpatient_claims.groupby('Provider').IsOutpatient.count().values

# Ratio of inpatient claims can be determined from the above - do we need a separate col for it?

In [ ]:
def binary_percentages(df, col, group1, group2):
    '''
    Adds a new column to a df that lists the ratio of one value from a
    binary column in the same df. (The ratio of the other value in the
    binary column is the inverse.)
    
    Arguments: A single dataframe.
    
    Output: None.
    
    Returns: Dataframe altered in place.
    '''
    by_provider = df.groupby(['Provider', col]
                            ).ClaimID.count().reset_index()

    colname = pd.DataFrame()
    colname[group1] = by_provider[by_provider[col] == 1].ClaimID.to_list()
    colname[group2] = by_provider[by_provider[col] == 0].ClaimID.to_list()
    
    newcolname1 = f'{group1}_perc'
    
    providers[newcolname1] = round(
        (colname[group1] / (colname[group1] + colname[group2])) * 100, 2)

In [ ]:
binary_percentages(claims, 'Gender', 'GenderZero', 'GenderOne')
binary_percentages(claims, 'RenalDisease', 'HasRenalDisease', 'NotRenalDisease')
binary_percentages(claims, 'HasDied', 'HasDied', 'NotDead')

In [ ]:
race_by_provider = \
    claims.groupby(['Provider', 'Race']).ClaimID.count().reset_index()

race = pd.DataFrame()
race['race1'] = race_by_provider[race_by_provider.Race == 1
                                ].ClaimID.to_list()
race['race0'] = race_by_provider[race_by_provider.Race == 0
                                ].ClaimID.to_list()
race['race3'] = race_by_provider[race_by_provider.Race == 3
                                ].ClaimID.to_list()
race['race5'] = race_by_provider[race_by_provider.Race == 5
                                ].ClaimID.to_list()

providers['RaceZero_perc']  = round(
    (race.race0 / np.sum(race, axis=1) * 100), 2)
providers['RaceThree_perc'] = round(
    (race.race3 / np.sum(race, axis=1) * 100), 2)
providers['RaceFive_perc']  = round(
    (race.race5 / np.sum(race, axis=1) * 100), 2)

In [ ]:
# DO WE WANT CHRONIC CONDITION RATIOS? WHY/WHY NOT?
# Ratio of inpatients with top 5 frequent chronic disease (from PotentialFraud)
# Ratio of outpatients with top 5 frequent chronic disease (from PotentialFraud)

In [ ]:
ratio_cols = providers.columns[providers.columns.str.contains('_perc')
                              ].to_list()
providers[ratio_cols].sample(5).iloc[:,-5:]

### Inpatients/Outpatiens

In [ ]:
binary_percentages(claims, 'IsOutpatient', 'IsOutpatient', 'Inpatient')

In [ ]:
# Whether the provider serves both in/out patients
# Based on providers.IsOutpatient_ratio, do we need this?

### Doctors

In [ ]:
binary_percentages(claims, 'HasAnyPhysician', 'HasAnyPhysician', 'HasNoPhysician')
binary_percentages(claims, 'HasAllPhysicians', 'HasAllPhysicians', 'HasSomePhysicians')

### Codes

### Money

### Time

### Location

### Size

## MISC/NOTES

In [ ]:
providers.columns#[providers.columns.str.contains('Chronic')].to_list()
providers.sample(10)

In [ ]:
# Ratio of attending physicians serving for different hospitals (75% threshold)
# Ratio of operating physicians serving for different hospitals (75% threshold)
# Ratio of other physicians serving for different hospitals (75% threshold)
# Ratio of inpatients going to different hospitals (75% threshold)
# Ratio of outpatients going to different hospitals (75% threshold)


# MBA, imbalanced data sets, classification modeling